Notebook was executed on kaggle server using GPU. Before executing the notebook have to add data on the notebook 
https://www.kaggle.com/zuozhaorui/celeba.

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn.functional as F
import os
import random
import shutil
import os
import cv2
import math

import matplotlib.image as mpimg
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFilter
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
# path starting with - '../input/' represents all the files that were uploaded on the server
# path startin with - './folder_name' represent all the folder and files created by us

In [ ]:
torch.cuda.get_device_name(0)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#### Function sample_data creates a  subset of main dataset for experimentation. In this case it is 100
def sample_data(source_folder_path, parent_dir):
    files_list = []
    for root, dirs, files in os.walk(source_folder_path):
        for file in files: 
            files_list.append(os.path.join(root, file))
            
    selected_files = random.sample(files_list, 100)
    directory = "dataset100"
    path = os.path.join(parent_dir, directory)
    os.mkdir(path)
    dest_path1 = path

    for src_path in selected_files:
        shutil.copy(src_path, os.path.join(dest_path1, os.path.basename(src_path)))
    return dest_path1

def create_directory(parent_dir, name):
    directory = name
    path = os.path.join(parent_dir, directory)
    os.mkdir(path)
    return path

#### Function move_multiple transfers a specific number of images from a source folder to destination folder.
def move_multiple(source_path, detination_path,number):
    files_list = []
    for root, dirs, files in os.walk(source_path):
        for file in files: 
            files_list.append(file)
        
    selected_files = random.sample(files_list, number)
    source_folder = source_path + "/"
    destination_folder = detination_path + "/"
    #print(selected_files)

    for file in selected_files:
        source = source_folder + file
        destination = destination_folder + file
        shutil.move(source,destination)
    return source_folder

#### Function move_all transfers all the images from source folder to destination folder
def move_all(source_path, detination_path):
    source_folder = source_path + "/"
    destination_folder = detination_path + "/"
    for file in os.listdir(source_folder):
        source = source_folder + file
        destination = destination_folder + file
        if os.path.isfile(source):
            shutil.move(source,destination)

In [ ]:
# get_cordinates helper function will take Length and Breadth of the image & generate one of the corner cordinates of patch image
# get_lengths_patch will return the patch length given the dimension of the image and cordinate of the corner of the patch

def get_cordinates(L, B, alpha):
    x = random.randint(0, math.floor(alpha * L))
    y = random.randint(0, math.floor(alpha * B))
    return x,y

def get_lengths_patch(L, x, beta):
    l = random.randint(math.floor(beta * L), L)
    while(x + l > L):
        l = random.randint(math.floor(beta * L), L)
    return l
# prepare_data_set method creates images by pasting the white patches of randomly generated sizes on the input images at random positions

def prepare_data_set(input_folder_path, output_folder_path, num_copies):
    #data_descripition_file = open(os.path.join(output_folder_path, 'data_description.csv'), "w")
    c=0
    for img_name in os.listdir(input_folder_path):
        c+=1
        if c%1000==0:
            print(c)
        for i in range(num_copies):
            image = Image.open(os.path.join(input_folder_path, img_name))
            L,B = image.size
            x,y = get_cordinates(L, B, 0.9)
            l = get_lengths_patch(L, x, 0.1 )
            b = get_lengths_patch(B, y, 0.1)
            patch_rect = np.ones((l,b,3),np.uint8)
            cv2.imwrite(os.path.join(output_folder_path, 'patch_image.png'), patch_rect)
            patch_rect = Image.open(os.path.join(output_folder_path, 'patch_image.png'))
            os.remove(os.path.join(output_folder_path, 'patch_image.png'))
            image.paste(patch_rect, (x,y))
            name = os.path.splitext(img_name)[0]
            image.save(os.path.join(output_folder_path, name + '.png'))
            #data_descripition_file.write(name + "," + str(l) + "," + str(b) + "," + str(x) + ","+str(y)+"\n")
    #data_descripition_file.close()

In [38]:
# seperate out 20k celeba images into a folder
os.mkdir('./train')
original = os.listdir("../input/celeba/img_align_celeba")
c =0
for i in original:
    c+=1
    if c%1000==0:
        print(c)
    if c < 20000:
        file = os.path.join("../input/celeba/img_align_celeba", i)
        shutil.copy(file, './train/')
    else:
       break


In [39]:
os.mkdir('./masked')

input_folder_path = "./train/" # unmasked original photos
output_folder_path = "./masked/" # masking photos saved in this folder

prepare_data_set(input_folder_path, output_folder_path, 1)

In [48]:

# randomly pick few images to test and visulaize
# remove them from the test and mask folder both
#os.mkdir('./prediction')

original = os.listdir("./masked/")
lt = np.random.choice(original,5,replace=False)
# same files are used that were used in auto encoder model
# for model comparison
#lt = ['000125.png','000139.png','000168.png','000178.png','000182.png']

c=0
for i in lt:
  print(c)
  c = c+1
  file = os.path.join("./masked/", i)
  file1 = os.path.join("./train/", i[:-3]+'jpg')
  shutil.copy(file, './prediction')
  # remove files from both train and mask folder
  os.remove(file) 
  os.remove(file1)

In [ ]:
# above created three folders train,mask and prediction
# were downloaded on local and also uiploaded on kaggle server as
# personal dataset so that do not have to create them again and again

# after uploading folder falls under input directory
# but while creating they are under output directory

In [49]:
path, dirs, files = next(os.walk("../input/train-mask-celeba/train/"))
file_count = len(files)
file_count

In [50]:
path, dirs, files = next(os.walk("../input/train-mask-celeba/masked/"))
file_count = len(files)
file_count

## CNN AE

In [51]:
# Load data using following class
class CustomDataSet(Dataset):
    def __init__(self, dir, transform):
        self.dir = dir
        self.transform = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])
        self.imgs = os.listdir(dir)
        self.imgs.sort()
        
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.dir, self.imgs[idx])
        image = Image.open(img_path).convert("RGB")
        img = self.transform(image)
        return (img,img)

In [52]:
# transform
transform = transforms.Compose( [transforms.Resize((64,64)),transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5]),transforms.ToTensor()])

# 'train' and 'valid' are folders with training and validation set
dataset = CustomDataSet('../input/train-mask-celeba/train/',transform=transform) # unmasked
dataset2 = CustomDataSet('../input/train-mask-celeba/masked/',transform=transform) # masked images

train_data = []
for i in range(len(dataset)):
    if i%1000==0:
        print(i)
    train_data.append([dataset2[i], dataset[i]])

In [53]:
k = 15 # test percent
tst_size = int(len(train_data)*k/100)
tst_size

In [55]:
# print i index of data3
p=(dataset2[6][0].cpu().detach().numpy())
plt.imshow(np.transpose(p, (1, 2, 0)))

In [56]:
# print i index of data3
p=(dataset[6][0].cpu().detach().numpy())
plt.imshow(np.transpose(p, (1, 2, 0)))

In [57]:
# similarly load labels and pass that in dataloader
trainloader = torch.utils.data.DataLoader(train_data[:-tst_size], batch_size=64, num_workers=2,shuffle=True)
valloader = torch.utils.data.DataLoader(train_data[len(train_data)-tst_size:], batch_size=64, num_workers=2,shuffle=True)
# testloader = torch.utils.data.DataLoader((dataset,label) ,batch_size=1,shuffle=True, num_workers=2)

First Image check

In [58]:
## First image check
for i,j in trainloader:
  break
imag_n = i[0].numpy()
plt.imshow(np.transpose(imag_n[0], (1, 2, 0)))

In [59]:
## First original image check
imag_n = j[0].numpy()

plt.imshow(np.transpose(imag_n[0], (1, 2, 0)))

Second image Check

In [60]:
for i,j in trainloader:
  break
imag_n = i[0].numpy()
plt.imshow(np.transpose(imag_n[0], (1, 2, 0)))

In [61]:
imag_n = j[0].numpy()

plt.imshow(np.transpose(imag_n[0], (1, 2, 0)))

In [62]:
model = nn.Sequential(
            nn.Conv2d(3,64,4,bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64,64,4,bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64,128,4,bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128,256,4,bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256,512,4,bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512,512,4, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.ConvTranspose2d(512,512, 4,bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128,64 , 4,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64,64 , 4,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4,bias=False),
            nn.ReLU(True),
            nn.Tanh()
            # image size 3 x 64 x 64
)

In [63]:
model.to(DEVICE)

In [64]:
loss_fn = nn.MSELoss()

In [66]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
samay = time()
epochs = 10

for e in range(epochs):
    epoch_loss = 0
    c=0
    #print(e)
    for images,clear in trainloader:
        print(c)
        images= images[0].to(DEVICE)
        clear= clear[0].to(DEVICE)

        ##  Forward pass
        # get output for the images
        output = model(images)
        
        ## Calculate Loss
        loss = loss_fn(output, clear)
        c=c+1
        ## Backward pass
        # Training pass- gradient zero initially
        optimizer.zero_grad()

        ## Backpropagation step 
        loss.backward()
        
        ## And optimizes its weights here
        optimizer.step()
        
        # loss.item() 
        epoch_loss += loss.item()
    else:     # it is executed when all the samles from training data has been processed and nothinig left at the end
        print("Epoch ",e," - Training loss: ", epoch_loss/len(trainloader))
    
    #---- TEST ----
    sahi = 0
    total = 0
    with torch.no_grad():
        print('test')
        for images1,clear1 in valloader:
            images1 = images1[0].to(DEVICE)

            output = model(images1)
  
            loss = loss_fn(output, images1)
            print(loss)
    # training time printed to check how many minutes it take to run each epoch  
    print("\nTraining Time (minutes) =",(time()-samay)/60)

In [67]:
## Predcition
# Prediction folder include all the images to be predictive 
# for now it have only 1 image
dataset2 = CustomDataSet('../input/prediction',transform=transform)
ader = torch.utils.data.DataLoader(dataset2, num_workers=2)

In [73]:
# predict images and save them
c=0
for i in ader:
    c+=1
    imag_n = i[0].numpy()
    #plt.imshow(np.transpose(imag_n, (1, 2, 0)))
    with torch.no_grad():
        images1 = i[0].to(DEVICE)
        output = model(images1)
        p = output.cpu().detach().numpy()
    #plt.imshow(np.transpose(p, (1, 2, 0)))
    plt.figure(figsize=(10,10))
    plt.subplot(len(ader),1,c)
    plt.imshow(np.transpose(p[0], (1, 2, 0)))
    name = str(c)+'.jpg'
    plt.savefig(name)

In [75]:
import os
co = os.listdir('../input/final-predict')

In [97]:
c = 0
p = 100
n = 4 # number of images uploaded
for ip in co:
    c =c+1
    p=p+1
    zz = os.path.join("../input/final-predict/", ip)
    img = Image.open(zz)
    #img = Image.open('../input/final-predict/001161.jpg')
    res = transforms.Resize((64,64)) # resize image
    img = res(img)
    convert_tensor = transforms.ToTensor()# convert resized image to tensor
    image_tensor = convert_tensor(img)
    
    # mask using pytorch
    mask = torch.ones(size=image_tensor.shape)
    start = int(image_tensor.shape[1] * 0.5)
    end = int(image_tensor.shape[1] * (0.7))
    mask[:,44:50, 22:40] = 0.0
    aa = torch.mul(image_tensor, mask)
    imag_n = aa.numpy()
    
    n_m = str(c)+'m.png'
   # plt.imshow(np.transpose(imag_n, (1, 2, 0)))
    plt.imsave(n_m,np.transpose(imag_n, (1, 2, 0)))

    po = np.expand_dims(aa, axis=0)
    po = torch.from_numpy(po)
    
    with torch.no_grad():
        images1 = po.to(DEVICE)
        output = model(images1)
        p = output.cpu().detach().numpy()
        #plt.imshow(np.transpose(p[0], (1, 2, 0)))

    mask = torch.ones(size=image_tensor.shape)
    aa[:,44:50, 22:40] = 0.8
    mask[:,44:50, 22:40] = torch.from_numpy(p[0][:,44:50, 22:40])
    aaa = torch.mul(aa, mask)
    imag_n = aaa.numpy()
    
    #plt.imshow(np.transpose(imag_n, (1, 2, 0)))
    
    plt.figure(figsize=(10,10))
    plt.subplot(n,1,c)
    plt.imshow(np.transpose(imag_n, (1, 2, 0)))
    name = str(c)+'p.jpg'
    plt.savefig(name)